In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks

In [ ]:
SEED = 42
IMG_SIZE = 300      
BATCH_SIZE = 32
EPOCHS = 18
LR = 1e-4

TRAIN_CSV = "/kaggle/input/cassava-leaf-disease-classification/train.csv"
TRAIN_DIR = "/kaggle/input/cassava-leaf-disease-classification/train_images"
TEST_DIR  = "/kaggle/input/cassava-leaf-disease-classification/test_images"
WEIGHT_B3 = "/kaggle/input/keras-efficientnetb3-noisy-student/noisy_student_efficientnet_b3.h5"

tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
train_df["image_id"] = train_df["image_id"].astype(str)
train_df["label"] = train_df["label"].astype(str)

In [ ]:
counts = train_df["label"].value_counts()
max_count = counts.max()
print("Before oversampling:\n", counts)

oversampled = [train_df]
for cls, cnt in counts.items():
    if cnt < max_count:
        need = max_count - cnt
        extra = train_df[train_df["label"] == cls].sample(need, replace=True, random_state=SEED)
        oversampled.append(extra)

train_balanced = pd.concat(oversampled).reset_index(drop=True)
print("After oversampling:\n", train_balanced["label"].value_counts())

In [ ]:
train_df_final, val_df = train_test_split(
    train_balanced,
    test_size=0.15,
    stratify=train_balanced["label"],
    random_state=SEED
)


In [ ]:
train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.05,
    horizontal_flip=True,
    brightness_range=(0.85, 1.15),
    fill_mode="reflect"
)

val_aug = ImageDataGenerator(rescale=1./255)
test_base = ImageDataGenerator(rescale=1./255)

train_gen = train_aug.flow_from_dataframe(
    train_df_final,
    directory=TRAIN_DIR,
    x_col="image_id",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_gen = val_aug.flow_from_dataframe(
    val_df,
    directory=TRAIN_DIR,
    x_col="image_id",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [ ]:
test_images = sorted(os.listdir(TEST_DIR))
test_df = pd.DataFrame({"image_id": test_images})

test_gen = test_base.flow_from_dataframe(
    test_df,
    directory=TEST_DIR,
    x_col="image_id",
    y_col=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode=None,
    shuffle=False,
    batch_size=BATCH_SIZE
)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(train_df["label"])   

cw = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=train_df["label"]
)

class_weight = {int(cls): weight for cls, weight in zip(classes, cw)}
print("class_weight:", class_weight)

In [ ]:
base = EfficientNetB3(
    include_top=False,
    weights=None,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base.load_weights(WEIGHT_B3)

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.4)(x)
out = Dense(5, activation="softmax")(x)
model = Model(base.input, out)

model.compile(
    optimizer=Adam(LR),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

In [ ]:
 chk = callbacks.ModelCheckpoint(
    "best_b3.h5", monitor="val_loss",
    save_best_only=True, verbose=1
)
es = callbacks.EarlyStopping(
    monitor="val_loss", patience=5,
    restore_best_weights=True
)
rlr = callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5,
    patience=2, min_lr=1e-6
)

In [ ]:
 steps_tr = int(np.ceil(train_gen.n / BATCH_SIZE))
steps_val = int(np.ceil(val_gen.n / BATCH_SIZE))

model.fit(
    train_gen,
    epochs=EPOCHS,
    steps_per_epoch=steps_tr,
    validation_data=val_gen,
    validation_steps=steps_val,
    class_weight=class_weight,
    callbacks=[chk, es, rlr],
    verbose=1
)

In [ ]:
tta_transforms = [
    ImageDataGenerator(rescale=1./255),
    ImageDataGenerator(rescale=1./255, horizontal_flip=True),
    ImageDataGenerator(rescale=1./255, rotation_range=15),
    ImageDataGenerator(rescale=1./255, zoom_range=0.1),
]

preds = np.zeros((test_gen.n, 5), dtype=float)

for i, aug in enumerate(tta_transforms):
    print(f"TTA {i+1}/{len(tta_transforms)}")
    tta_gen = aug.flow_from_dataframe(
        test_df,
        directory=TEST_DIR,
        x_col="image_id",
        y_col=None,
        target_size=(IMG_SIZE, IMG_SIZE),
        class_mode=None,
        shuffle=False,
        batch_size=BATCH_SIZE
    )
    preds += model.predict(tta_gen, verbose=1)

preds /= len(tta_transforms)
final_labels = preds.argmax(axis=1)

In [ ]:
submission = pd.DataFrame({
    "image_id": test_df["image_id"],
    "label": final_labels
})
submission.to_csv("submission.csv", index=False)

submission.head()